In [1]:
!/usr/bin/env python -m pip install selenium pandas


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import time
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [5]:
firefox_options = webdriver.FirefoxOptions()
driver = webdriver.Remote(command_executor="http://localhost:4444/wd/hub", options=firefox_options)

In [11]:
routes = []

In [12]:
driver.get(f"https://2gis.ru/moscow/search/Детские+сады/rubricId/237")
while True:
    routes.extend(el.find_element(By.CLASS_NAME, "_1rehek").get_attribute('href') for el in driver.find_elements(By.CLASS_NAME, "_1kf6gff") if el.find_element(By.CLASS_NAME, "_oqoid").text == "Детские сады")
    driver.execute_script("document.querySelector('div._n5hmn94:last-child').click();")
    time.sleep(1.5)

JavascriptException: Message: TypeError: document.querySelector(...) is null
Stacktrace:
@https://2gis.ru/moscow/search/%D0%94%D0%B5%D1%82%D1%81%D0%BA%D0%B8%D0%B5%2B%D1%81%D0%B0%D0%B4%D1%8B/rubricId/237:2:16
@https://2gis.ru/moscow/search/%D0%94%D0%B5%D1%82%D1%81%D0%BA%D0%B8%D0%B5%2B%D1%81%D0%B0%D0%B4%D1%8B/rubricId/237:3:8


In [10]:
len(routes)
routes[0]

'https://2gis.ru/moscow/firm/4504127908474124?stat=eyJwYXJlbnRUYWJJZCI6ImY4MTU1NzA1LTIzYjMtNGU3YS1hMTI0LTgyNWU1MDU5Nzg0YiIsImZvcmtFdmVudE9yZGluYWwiOjIsInVpRWxlbWVudCI6eyJuYW1lIjoicGxhY2VDYXJkTWluaSIsIm93bmVyTmFtZSI6InNlYXJjaFJlc3VsdHNMaXN0IiwicG9zaXRpb24iOjN9LCJwbGFjZUl0ZW0iOnsiZW50aXR5Ijp7ImlkIjoiNDUwNDEyNzkwODQ3NDEyNCIsInR5cGUiOiJicmFuY2giLCJzZWdtZW50SW5mbyI6eyJiYXNlTG9jYWxlIjoicnVfUlUiLCJzZWdtZW50SWQiOiIzMiJ9fSwiZ2VvUG9zaXRpb24iOnsibG9uIjozNy42NDA5NTMsImxhdCI6NTUuNzU0OTk2fSwiaHlicmlkRW50aXRpZXMiOlt7ImlkIjoiNDUwNDIzNTI4MjczNTk3NiIsInR5cGUiOiJidWlsZGluZyJ9XSwiYWRzU2VhcmNoIjpmYWxzZSwibWFpblJ1YnJpYyI6IjIzNyIsImlzRGVsZXRlZCI6ZmFsc2UsIm9yZyI6IjQ1MDQxMzY0OTgzMjczMjMiLCJjb250ZXh0UnVicmljIjoiMjM3IiwicmVzdWx0Q2xhc3MiOjIsInBvc2l0aW9uIjozfSwic2VhcmNoUmVzdWx0c0xpc3QiOnsic2l6ZSI6MzkyOSwic2VhcmNoVHlwZSI6MiwiZmlyc3RSZXN1bHRDbGFzcyI6MiwiY29udGV4dFJ1YnJpY3MiOlsiMjM3Il0sImhhc0dlb1Jlc3RyaWN0aW9uIjpmYWxzZSwicGFydGlhbFJlc3VsdCI6ZmFsc2UsImFkdmVydGlzZXJzIjpbeyJvcmciOiI3MDAwMDAwMTAxODI5Mzk2MiIsImNvbnRleHRSd

In [15]:
from urllib.parse import unquote

FIRM_ID_REGEX = re.compile(r"firm/(\d+)")
COORDS_REGEX = re.compile(r"points\/.?(?P<lon>\d+\.\d+),(?P<lat>\d+\.\d+);")

kindergartens = []
for route_url in routes:
    firm_id = FIRM_ID_REGEX.search(route_url)[1]
    driver.get(route_url)
    title = driver.find_element(By.XPATH, "//h1[@class='_tvxwjf']/span").text
    href = driver.find_element(By.XPATH, "//div[@class='_vaj62s'][last()]/a[@class='_1qhm93s']").get_attribute("href")
    coords = COORDS_REGEX.search(unquote(href)).groupdict()
    kindergartens.append(dict(firm_id=firm_id, title=title, **coords))

In [16]:
import json
with open("./kindergartens.json", 'w') as f:
    json.dump(kindergartens, f)

In [17]:
import pandas as pd
df = pd.DataFrame.from_records(kindergartens)
df.head()

,firm_id,title,lon,lat
0,4504127908474124,Детский сад №395,37.640953,55.754996
1,4504127908786633,Школа №641 им. С. Есенина с дошкольным отделением,37.75183,55.705557
2,4504128908394258,Школа №1290 с дошкольным отделением,37.763389,55.798272
3,4504127908351744,Школа им. Маршала В.И. Чуйкова с дошкольным от...,37.764193,55.71196
4,4504127908490635,Школа №1164,37.574328,55.796348


In [18]:
df.lon = pd.to_numeric(df.lon)
df.lat = pd.to_numeric(df.lat)

In [19]:
import redis
R = redis.Redis(host='localhost')

In [20]:
for index, row in df.iterrows():
    data = row.to_dict()
    lon = data.pop("lon")
    lat = data.pop("lat")
    R.geoadd("organization:kindergarten", (lon, lat, json.dumps(data)))